# Unsupervised Predict: Explore Data Science Academy
## Movie Recommender Engine 

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### Importing Libraries

In [ ]:
# Install packages here
# Packages for data processing
import numpy as np
import pandas as pd
import seaborn as sns
import math
import random
import pickle
import json
import matplotlib.pyplot as plt
import re
import warnings

from datetime import datetime
from scipy.sparse import csr_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse.linalg import svds
from sklearn.preprocessing import MinMaxScaler, StandardScaler

from wordcloud import WordCloud, STOPWORDS

# !pip install scikit-surprise
from surprise import Reader, Dataset
from surprise.model_selection import cross_validate
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from surprise import NormalPredictor
from surprise import SVDpp, SVD
from surprise import NMF
from surprise import KNNBasic, KNNWithMeans, KNNWithZScore, KNNBaseline
from surprise import BaselineOnly, SlopeOne, CoClustering
from surprise.accuracy import rmse
from surprise import accuracy
from surprise.model_selection import train_test_split
from surprise.model_selection import GridSearchCV

warnings.filterwarnings("ignore")
sns.set_style('dark')
%matplotlib inline

In [ ]:
#Ignore warnings
import warnings
warnings.simplefilter(action='ignore')

#Install Prerequisites
import sys
#!{sys.executable} -m pip install scikit-learn scikit-surprise
#!pip install git+https://github.com/gbolmier/funk-svd

# Exploratory Data Analysis
import pickle
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Data Preprocessing
import random
from time import time

from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.ticker import NullFormatter
from sklearn.preprocessing import StandardScaler
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

# Models
from surprise import Reader, Dataset
from surprise import SVD, NormalPredictor, BaselineOnly, NMF, SlopeOne, CoClustering
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

# Performance Evaluation
from surprise import accuracy
from sklearn.metrics import mean_squared_error
from surprise.model_selection import GridSearchCV, cross_validate, train_test_split

# Display
%matplotlib inline
sns.set(font_scale=1)
sns.set_style("white")
pd.set_option('display.max_columns', 37)
from wordcloud import WordCloud 
import warnings
warnings.filterwarnings('ignore')
from IPython.display import display_html 
from IPython.core.display import HTML
from collections import defaultdict
#import datetime
import re
import squarify
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
# Visualisation
import matplotlib.pyplot as plt 
import seaborn as sns
sns.set_style("darkgrid")
import plotly.offline as py
import plotly.express as px
import plotly.graph_objects as go
from plotly.offline import init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
py.init_notebook_mode(connected = True)
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

### Reading in Data

In [ ]:
#Import datasets
sample_submission = pd.read_csv('/kaggle/input/edsa-movie-recommendation-challenge/sample_submission.csv')
movies = pd.read_csv('/kaggle/input/edsa-movie-recommendation-challenge/movies.csv')
imdb_data = pd.read_csv('/kaggle/input/edsa-movie-recommendation-challenge/imdb_data.csv')
genome_scores = pd.read_csv('/kaggle/input/edsa-movie-recommendation-challenge/genome_scores.csv')
genome_tags = pd.read_csv('/kaggle/input/edsa-movie-recommendation-challenge/genome_tags.csv')
train = pd.read_csv('/kaggle/input/edsa-movie-recommendation-challenge/train.csv')
test = pd.read_csv('/kaggle/input/edsa-movie-recommendation-challenge/test.csv')
tags = pd.read_csv('/kaggle/input/edsa-movie-recommendation-challenge/tags.csv')
links = pd.read_csv('/kaggle/input/edsa-movie-recommendation-challenge/links.csv')

In [ ]:
movies.head()

In [ ]:
sample_submission.head()

### Data Preprocessing & Preparation

Get an indication of the size of the different datasets

In [ ]:
# Create a list consisting of the names of our dataframes
dfs = [train, test, genome_scores, genome_tags, imdb_data, links, movies, tags]
# Declare a list of the names of the imported datasets
df_names = ['train', 'test', 'genome_scores', 'genome_tags',
            'imdb_data', 'links', 'movies', 'tags']
dfs_dict = {}  # declare an empty dictionary in preparation for dataframe view
for name, data in zip(df_names, dfs):  # iterate over the zipped lists 
    dfs_dict[name] = [data.shape[0], data.shape[1]] # retrieve row and column sizes
    # convert dictionary into dataframe
    df_prop = pd.DataFrame(dfs_dict,
                          index=['rows', 'columns']).transpose()
df_properties = df_prop.sort_values(by='rows', ascending=False) # Arrange dataframes in descending order of samples (rows)

df_properties  # View shapes of dataframes

In [ ]:
# Obtain a feel of nature of data present in train dataset
train.head()

Understand what data types are present within our two main datasets (train and test)

In [ ]:
train.info()

In [ ]:
test.info()

Establish how much data is missing

train_missing = pd.DataFrame(train.isnull().sum())
train_missing

In [ ]:
test_missing = pd.DataFrame(test.isnull().sum())
test_missing

In [ ]:
g_scores_missing = pd.DataFrame(genome_scores.isnull().sum())
g_scores_missing

In [ ]:
g_tags_missing = pd.DataFrame(genome_tags.isnull().sum())
g_tags_missing

In [ ]:
movies_missing = pd.DataFrame(movies.isnull().sum())
movies_missing

In [ ]:
links_missing = pd.DataFrame(links.isnull().sum())
links_missing

In [ ]:
tags_missing = pd.DataFrame(tags.isnull().sum())
tags_missing

In [ ]:
imdb_missing = pd.DataFrame(imdb_data.isnull().sum())
imdb_missing

The above outputs show that there is no missing data from our primary datasets (train and test). There is a negligible proportion of missing values within the links and tags datasets. A considerable number of missing entries occurs within the imdb dataset which would require further investigation. Let's determine what percentage of data is missing in each of this dataset's columns. 

In [ ]:
# Calculating the percentage of missing values in each column of the imdb dataset
total = imdb_data.isnull().sum().sort_values(ascending=False)
percent_1 = imdb_data.isnull().sum()/imdb_data.isnull().count()*100
percent_2 = (round(percent_1, 1)).sort_values(ascending=False)
missing_data = pd.concat([total, percent_2],
                         axis=1, keys=['Total', '(%) missing'])
missing_data['(%) missing'].plot(kind='barh')
plt.xlabel('(%) Missing Values')
plt.ylabel('Columns with Missing Values')
plt.title('Percentage of Missing Values per Column')
plt.show()

It is clear from the above chart that there is a large proportion of missing entries across all variables (barring moviedId) with the effect that this precludes the use of this dataset for modeling purposes.

In [ ]:
# Ascertain how many users and movies there are and whether there are any duplications (within the train dataset)
users = len(train.userId.unique())
items = len(train.movieId.unique())
print('There are {} unique users and {}\
 unique movies train dataset with {} duplicated entries'.format(users, items, train[train.duplicated()].count().sum()))

In [ ]:
# Merge dataframes into one consolidated dataframe in preparation for more advanced EDA
# Merge train (users) and movies datasets on movieId column
train_movies_df = pd.merge(train,
                           movies,
                           how='left',
                           on='movieId')

# Perform a further merge with imdb_data on movieId column
movies_metadata_df = pd.merge(train_movies_df,
                              imdb_data,
                              how='left',
                              on='movieId')

movies_metadata_df.head()

In [ ]:
# Calculate average rating for each movie and arrange from highest to lowest
movies_ranking = movies_metadata_df[['title','rating']].groupby('title').mean().sort_values('rating', ascending=False)
movies_ranking.head()

Keep in mind that the above average ratings do not take movie popularity into account, in other words the number of people who have watched the movie. We will factor this in later and calculate weighted ratings. 

### EDA

Discovery phase and data understanding

**Most common Genres**

In [ ]:
"""
# Create dataframe containing only the movieId and genres
movies_genres = pd.DataFrame(df_movies[['movieId', 'genres']],
                             columns=['movieId', 'genres'])

# Split genres seperated by "|" and create a list containing the genres allocated to each movie
movies_genres.genres = movies_genres.genres.apply(lambda x: x.split('|'))

# Create expanded dataframe where each movie-genre combination is in a seperate row
movies_genres = pd.DataFrame([(tup.movieId, d) for tup in movies_genres.itertuples() for d in tup.genres],
                             columns=['movieId', 'genres'])

movies_genres.head()
"""

In [ ]:
"""
# Plot the genres from most common to least common
plot = plt.figure(figsize=(15, 10))
plt.title('Most common genres\n', fontsize=20)
sns.countplot(y="genres", data=movies_genres,
              order=movies_genres['genres'].value_counts(ascending=False).index,
              palette='Reds_r')
plt.show()
"""

Determine how the ratings are distributed across the rating scale (from 0.5 to 5.0)

In [ ]:
print (f'Average rating in the dataset: {np.mean(train["rating"])}')

with sns.axes_style('white'):
    g = sns.factorplot("rating", data=train, aspect=2.5, kind='count')
    g.set_ylabels("Total number of ratings")

The mean rating of approx 3.5 shows that users typically lean towards more favourable ratings and are not excessively harsh in their criticism. There are relatively few ratings below 2.0. An alternative explanation could be that users tend to only rate those movies that they enjoy. Many users might terminate an unenjoyable movie mid-stream without bothering to give it a rating. I've personally walked out of a cinema just 15 minutes into the showing of a junk movie, figuring I can make better use of my valuable time and simply deeming the purchase of my movie ticket as a sunk cost. The mode ranking in the chart above is 4.0. It appears that users tend to provide whole-number ratings (e.g. 4.0) as opposed to fractional ratings (e.g. 3.5). 

Establish how many ratings each movie has received to get an indication of movie popularity.

In [ ]:
movies_ranking['No_of_ratings'] = movies_metadata_df.groupby('title')['rating'].count()

In [ ]:
movies_ranking.sort_values(by=['No_of_ratings', 'rating'], ascending=False).head(15)

The above table shows that the most famous movies of our time have received the highest number of ratings, giving an indicatio of the close connection between popularity and quality of movie.

In [ ]:
# Set plot size
sns.set(rc={'figure.figsize':(12,9)})

# Plot Number of rating for every rating category.
sns.scatterplot(x='rating', y='No_of_ratings', data=movies_ranking)
plt.title('Number of ratings per average rating per movie')
plt.xlabel('Rating')
plt.ylabel('Number of ratings')
plt.show()

The above plot shows that movies that receive a higher number of ratings also tend to receive higher rating scores, lending further credence to the belief that popular movies (high number of views) are more highly rated. In the plot below, it can be seen that movies with more than 100 ratings obtain a rating of 3.5 as their mode. 

In [ ]:
# Average rating of movies in the dataset
avg_rating = train.groupby('movieId')['rating'].mean()

# Plotting the results
plt.figure(figsize=(12,10))
avg_rating.plot(kind='hist')
plt.ylabel('Frequency')
plt.xlabel('Movie Rating')
plt.title('Average ratings of movies with 100 or more viewers')
plt.show()

It is common knowledge that the identity of the director plays a large role in movie popularity and ratings. The table below shows the correlation between ratings count and rating (score) for each movie director. Those directors whose movies have received the most number of ratings typically also get higher rating scores for their movies.   

In [ ]:
best_director = pd.DataFrame(movies_metadata_df.groupby('director')['rating'].mean().
                             sort_values(ascending=False))
best_director['No_of_ratings'] = movies_metadata_df.groupby('director')['rating'].count()
best_director.sort_values(by=['No_of_ratings', 'rating'], ascending=False).head(10)

The scatterplot below provides a visual indication of the correlation between number of ratings and rating scores for directors. 

In [ ]:
# Set plot size
sns.set(rc={'figure.figsize':(12,9)})

sns.scatterplot(x = 'rating', y = 'No_of_ratings', data = best_director).set_title('Number of ratings per average rating per director')
plt.xlabel('Ratings')
plt.ylabel('Number of Ratings')
plt.show()

#### Extracting insights from release dates

In [ ]:
# Extract release date (year) from movie titles

# Use regular expression to extract year from paranthesis in movie title 
movies["year"] = movies.title.str.extract("\((\d{4})\)", expand=True)
# Remove parentheses so only year is extracted
movies["year"] = movies.year.str.extract("(\d\d\d\d)", expand=True)
# Remove year from ‘title’ column
movies["title"] = movies.title.str.replace("(\(\d\d\d\d\))", "")
# Strip any ending whitespace characters
movies["title"] = movies["title"].apply(lambda x: x.strip())

In [ ]:
# Remove pipe character in genres column of movies dataset
movies['genres'] = movies['genres'].str.replace('|',' ')

# Remove pipe character from title_cast column in imdb dataset
imdb_data['title_cast'] = imdb_data['title_cast'].str.replace('|',' ')
imdb_data['plot_keywords'] = imdb_data['plot_keywords'].str.replace('|',' ')

In [ ]:
imdb_data.head()

In [ ]:
movies.head(10)

In [ ]:
# map movie title to movieId:
Mapping_file = dict(zip(movies.title.tolist(), movies.movieId.tolist()))

In [ ]:
# Consolidate movies and imdb dataframes 
metadata = pd.merge(movies, imdb_data, on='movieId', how='left')
# Calculate the number of movies released each year
num = metadata.groupby('year').count()
plt.figure(figsize=(20,10))
plt.plot(num.index, num['budget'])
plt.xlabel("years", size=25)
plt.xticks(rotation='vertical')
plt.ylabel('No. of Movies', size=25)
plt.title('Number of Movies Released By year', size=25)
plt.show()
#metadata.head()

To obtain further insight into the increasing number of movies since 2010, the wordplot below gives a breakdown of the most prominent years in this period.

In [ ]:
# Calculate number of ratings by year 
year_corpus = metadata['year'].value_counts()
# Generating the wordcolud
year_wordcloud = WordCloud(background_color='white', height=2000, width=4000).generate_from_frequencies(year_corpus)
plt.figure(figsize=(16,8))
plt.imshow(year_wordcloud)
plt.axis('off')
plt.show()

In [ ]:
# Create 'year' df that averages release year for each user
years = pd.merge(train, movies, on='movieId')[['userId','year']].dropna()
years['year'] = years['year'].astype('int64')
years.groupby('userId').mean()

# Set plot size
sns.set(rc={'figure.figsize':(12,9)})

plt.figure(figsize=(12,10))
years['year'].plot(kind='hist')
plt.ylabel('Frequency')
plt.xlabel('Average release year per user')
plt.title('Distribution of release years for movies rated by users')
plt.show()

The above plot shows how rating counts are distributed by decade with users showing a preference for movies from certain decades. Next we extract the year from the timestamp column within the train dataset in order to get an indication of the distribution of rating counts by year.

In [ ]:
train['datetime_of_rating'] = train['timestamp'].apply(lambda x: datetime.fromtimestamp(x))
train['year_rated'] = train['datetime_of_rating'].dt.year
train_temp = train.drop(['timestamp', 'datetime_of_rating'], axis = 1)
train_temp.head()

In [ ]:
# Group ratings by year and derive an annual rating count
year_rated_df = train_temp[['year_rated','rating']].groupby('year_rated').count()

# Set plot size
sns.set(rc={'figure.figsize':(12,9)})

sns.scatterplot(x = 'year_rated', y = 'rating', data = year_rated_df.reset_index()).set_title('Number of ratings per average rating per director')
plt.xlabel('Year rated')
plt.ylabel('number of ratings')
plt.show()

The above plot shows that ratings were only collected from 1995 onwards. There is no clear correlation between rating counts and year in this time interval. The plot below uses a bar format to show the number of movie releases per year in descending order for the past 50 years.

In [ ]:
#Plotting total amount of movies released in each year using a count plot.
figure= plt.subplots(figsize=(15, 5))
axes=sns.countplot(x=movies['year'], order = movies['year'].value_counts()[0:50].index,color='blue')
axes.set_title('Total movies released per year',fontsize=19)
plt.xticks(rotation=90)
plt.show()

The above graph shows an increasing trend in movie releases since 1995. 

Let's now get an indication as to how rating counts vary by day of week

In [ ]:
# Convert the timestamp values into datetime format
train['timestamp'] = pd.to_datetime(train['timestamp'], unit='ms')
train.head()

In [ ]:
# Extract the day of the week from the newly-formatted timestamp column
train['day_of_week'] = train['timestamp'].dt.dayofweek
days = {0:'Mon',1:'Tue',2:'Wed',3:'Thur',4:'Fri',5:'Sat',6:'Sun'}
train['day_of_week'] = train['day_of_week'].apply(lambda x: days[x])
train.tail()

In [ ]:
# Plot number of ratings by day of week
train['day_of_week'].value_counts().plot(kind='bar')
plt.title('Ratings per day of the week')
plt.xlabel('Day of the week')
plt.ylabel('Proportion of ratings created/collected')
plt.show()

It is evident from the above that Saturdays and Sundays account for the highest number of ratings which can most likely be attributed to the fact that these are the days which people do not attend school, college, their place of work, etc.

#### Exploring Genres and Tags

Let's asses how prevalent each of the different genres are within our consolidated dataset

In [ ]:
# Combine genres into single string
metadata['genres'] = metadata['genres'].astype('str')
genre_corpus = ' '.join(metadata['genres'])
#Generate a list of stopwords to exclude from WordCloud
stopword = ['no genres', 'no', 'genres', 'genre', 'listed']
# Create the wordcolud
genre_wordcloud = WordCloud(stopwords=stopword, background_color='white', height=2000, width=4000).generate(genre_corpus)
plt.figure(figsize=(16,8))
plt.imshow(genre_wordcloud)
plt.axis('off')
plt.show()

Comedy, Drama, Sci-Fi and Romance constitute the most prevalent genres. Action, Adventure, Crime and Thriller also account for genres which are often watched.

In [ ]:
sns.violinplot(x = 'runtime', data = metadata,).set_title('Distribution of Movie Duration')
plt.show()

In [ ]:
print('runtime mean: ', metadata['runtime'].mean())
print('runtime standard deviation: ', metadata['runtime'].std())

The average movie runtime length is 101 minutes with the average deviation being approximately 30 minutes either side of this figure. Further analysis and modeling can reveal the extent to which runtime is a significant determinant of the rating given to a movie.

Movie tags are explored below. Each tag is given a relevance rating between 0 and 1 to indicate the extenmt to which the tag appropriately describes the movie.

In [ ]:
genome_tags = pd.read_csv('/kaggle/input/edsa-movie-recommendation-challenge/genome_tags.csv')

In [ ]:
genome_df = pd.merge(genome_scores,genome_tags, how='left',on='tagId')

In [ ]:
genome_df.head()

With most movies possessing multiple tags, let's sort each movie's tags in order of most to least relevant and establish how important a relevant tag is in determining a movie's rating.

In [ ]:
genre_tag_df = genome_df.merge(movies, on = 'movieId', how = 'outer')

In [ ]:
genre_tag_df.head()

In [ ]:
genre_tag_df.dropna(axis = 0, inplace = True) # Drop movies that don't have a tag
genre_tag_df['tag_rank'] = genre_tag_df.groupby("movieId")["relevance"].rank(method = "first", 
                                                                                       ascending = False).astype('int64')

In [ ]:
# Visualise sorted tags for a selected movie ('Jumanji')
genre_tag_df[genre_tag_df.title == 'Jumanji'][['movieId','title','tag','relevance',
                                                               'tag_rank']].sort_values(by = 'relevance'
                                                                                              , ascending = False).head(10)

The tag that best defines the above movie is 'adventure'. There are several other tags also considered highly relevant, such as: 'children', 'fantasy' and 'jungle'. 

### Modeling

Having been given a choice to select either a content-based or collaborative-based algorithm, I have decided in favour of the latter. This technique is well-established, has been adopted by Netflix and performs well given a large database of users and a high number of historical ratings given to movies. Although it suffers from the 'cold-start' problem when a new user joins a movie-streaming service, this can be partly overcome by recommending popular movies. An advantage of the collaborative-based technique is the diversity of recommendations provided to users across a range of genres, actors, directors and release-year.   

In [ ]:
# Limit training sample to 500 000 rows 
train_sample = train.sample(n = 500000, replace = False)
# Drop the timestamp column since it is not needed
data = train_sample.drop('timestamp', axis = 1)
# Define the Reader object by specifying the rating scale range in the dataset
reader = Reader(rating_scale=(0.5, 5), line_format='user item rating') 
# Load the dataset 
data = Dataset.load_from_df(data[['userId', 'movieId', 'rating']], reader)
# Split dataset into train and validation sets
train_set, val_set = train_test_split(data, test_size=0.15, random_state=42)

Normal Predictor
Under the assumption of a normal distribution in the training set, this algorithm predicts a rating for each movie. 
An RMSE of 1.45 is returned - not a great score but, at least, it's a start.

In [ ]:
np_test = NormalPredictor()
np_test.fit(train_set)
predictions = np_test.test(val_set)
# Calculate RMSE
np_rmse = accuracy.rmse(predictions)

NMF

NMF is a collaborative filtering algorithm whch makes use of non-negative matrix factorization to output predicted ratings for missing values. As seen below, it has improved out RMSE score to 1.07.

In [ ]:
nmf = NMF()
nmf.fit(train_set)
predictions = nmf.test(val_set)
# Calculate RMSE
nmf_rmse = accuracy.rmse(predictions)

SlopeOne

The SlopeOne algorithm utilises linear regression to solve the data sparsity problem. Owing to the simplistic nature of the algorithm, we can see that the RMSE returned (1.16) is a long way off our target of 0.85.

In [ ]:
slo = SlopeOne()
slo.fit(train_set)
predictions = slo.test(val_set)
# Calculate RMSE
slo_rmse = accuracy.rmse(predictions)

BaselineOnly

Making use of either 'stochastic gradient descent' (SGD) or 'alternating least squares' (ALS), the Baseline Only algorithm predicts a baseline estimate rating for a given user and movie. With a returned RMSE score of 0.92, it looks like we're making good progress. 

In [ ]:
bsl_choice= {'method': 'sgd','n_epochs': 40}
blo = BaselineOnly(bsl_options=bsl_choice)
blo.fit(train_set)
predictions = blo.test(val_set)
# Calculate RMSE
blo_rmse = accuracy.rmse(predictions)

Co-Clustering

In similar manner to KMeans, the Co-clustering algorithm assigns data points to clusters using a straightforward optimization method. The RMSE achieved is 1.06.

In [ ]:
coc= CoClustering(random_state=42)
coc.fit(train_set)
predictions = coc.test(val_set)
# Calculate RMSE
coc_rmse = accuracy.rmse(predictions)

#### Singular Value Decomposition (SVD)

The Singular Value Decomposition algorithm is a matrix factorization technique which reduces the number of features of a dataset. In the matrix structure, each row represents a user and each column represents a movie. The matrix elements are ratings that users give to movies. A basic SVD model with limit hyperparameter tuning returns a score of 0.92.

In [ ]:
# Base algorithm
algo = SVD(n_epochs= 50, init_std_dev=0.02, n_factors=200)

# Fit to trainset
algo.fit(train_set)
 
# Make predictions on the validation dataset
predictions_svd1 = algo.test(val_set)
algo_rmse = accuracy.rmse(predictions_svd1)

In [ ]:
#predictions_svd1

In [ ]:
pred = pd.DataFrame(predictions_svd1)
pred

In [ ]:
pred=pred.rename(columns={'uid':'userId', 'iid':'movieId','est':'rating'})
pred.drop(['r_ui','details'],axis=1,inplace=True)

In [ ]:
pred.head()

In [ ]:
"""
svdpp_model = SVDpp(n_epochs=50,n_factors=400,init_std_dev=0.001,random_state=42, verbose=True)
svdpp_model.fit(trainset)
svdpp_predictions = svdpp_model.test(testset)
svdpp_rmse = accuracy.rmse(svdpp_predictions)
"""

In [ ]:
svdpp_model = SVDpp(n_epochs=20,n_factors=400,init_std_dev=0.001,random_state=42, verbose=True)
svdpp_model.fit(train_set)
svdpp_predictions = svdpp_model.test(val_set)
svdpp_rmse = accuracy.rmse(svdpp_predictions)

In [ ]:
"""
# Define search grid
param_grid = {'n_epochs': [40, 45,50], 'init_std_dev' : [0.02,0.05], 'n_factors' : [200,250]}

# Instatiate gridsearch instance
gs = GridSearchCV(SVD, param_grid, measures=['rmse'], cv=2)

# Run gridsearch
gs.fit(data)

# best RMSE score
print(gs.best_score['rmse'])

# combination of parameters that gave the best RMSE score
print(gs.best_params['rmse'])
"""

In [ ]:
"""
svdpp_model = SVDpp(n_epochs=50,n_factors=160,lr_all=0.0085,reg_all=0.02,init_std_dev=0.01,random_state=42, verbose=True)
svdpp_model.fit(train_set)
svdpp_predictions = svdpp_model.test(val_set)
svdpp_rmse = accuracy.rmse(svdpp_predictions)
"""

In [ ]:
rating_scale = Reader(rating_scale=(0.5, 5))
#train_df = Dataset.load_from_df(train.drop('timestamp', axis=1), rating_scale)
train_df = Dataset.load_from_df(train[['userId', 'movieId', 'rating']], rating_scale)

# Training and validation set split for hypertuning
train_set, val_set = train_test_split(train_df,
                                      test_size=0.008,
                                      random_state=42)

# Modelling of the SVD hypertuning
svd_algo_hyper = SVD(n_factors=160, 
                     lr_all=0.0085,
                     reg_all=0.02,
                     n_epochs=20,
                     init_std_dev=0.01)
svd_algo_hyper.fit(train_set)

# Predicting on the validation set
svd_hyper_predictions = svd_algo_hyper.test(val_set)

# Convert the predictions to dataframe
#test = pd.DataFrame(predictions)
accuracy.rmse(svd_hyper_predictions)

# Dictionary for the data to log for the SVD tuned model
#params = {'model_name': 'SVD_Tuned'}
#metrics = {'RMSE': accuracy.rmse(svd_hyper_predictions)}

# Log the parameters and results for the SVD tuned model
#experiment.log_parameters(params)
#experiment.log_parameters(metrics)
# End the experiment for the SVD tuned experiment
#experiment.end()

The RMSE of 0.93 achieved with this un-tuned SVD is not bad. It is not far off from the targeted RMSE value of 0.85. With some hyper-parameter tuning let's see how we can reduce this error metric further. 

In [ ]:
"""
rating_scale = Reader(rating_scale=(0.5, 5))
#train_df = Dataset.load_from_df(train.drop('timestamp', axis=1), rating_scale)
train_df = Dataset.load_from_df(train[['userId', 'movieId', 'rating']], rating_scale)

# Training and validation set split for hypertuning
train_set, val_set = train_test_split(train_df,
                                      test_size=0.008,
                                      random_state=42)

# Modelling of the SVD hypertuning
svd_algo_hyper = SVD(n_factors=160, 
                     lr_all=0.0085,
                     reg_all=0.02,
                     n_epochs=20,
                     init_std_dev=0.01)
svd_algo_hyper.fit(train_set)

# Predicting on the validation set
svd_hyper_predictions = svd_algo_hyper.test(val_set)

# Convert the predictions to dataframe
#test = pd.DataFrame(predictions)
accuracy.rmse(svd_hyper_predictions)

# Dictionary for the data to log for the SVD tuned model
#params = {'model_name': 'SVD_Tuned'}
#metrics = {'RMSE': accuracy.rmse(svd_hyper_predictions)}

# Log the parameters and results for the SVD tuned model
#experiment.log_parameters(params)
#experiment.log_parameters(metrics)
# End the experiment for the SVD tuned experiment
#experiment.end()
"""

Our tuned algorithm gives an impressive RMSE score of 0.78. Let's make use of this tuned SVD algorithm to predict on our test set and see how it scores on the Kaggle public leaderboard. 

In [ ]:
pred_svd_hyper = [svd_algo_hyper.predict(row.userId,
                                         row.movieId) for idx,row in test.iterrows()]

# Converting the predictions to a dataframe
test_pred_svd_hyper = pd.DataFrame(pred_svd_hyper)

### Submission dataset

In [ ]:
# Rename the fields in the prediction dataframe
test_pred_svd_hyper.drop(['r_ui', 'details'], axis=1, inplace=True)
test_pred_svd_hyper = test_pred_svd_hyper.rename(columns={'uid':'userId',
                                                          'iid':'movieId',
                                                          'est':'rating'})
test_pred_svd_hyper.head()

In [ ]:
# Concatenate each userId and movieId into a single Id column for submission
test_pred_svd_hyper['Id'] =  test_pred_svd_hyper['userId'].astype(str).str.zfill(1) + '_' + test_pred_svd_hyper['movieId'].astype(str).str.zfill(1)

In [ ]:
svd_hyper_predictions = test_pred_svd_hyper[['Id','rating']]
svd_hyper_predictions.head()

In [ ]:
svd_hyper_predictions.to_csv('kaggle_submission7.csv', index=False)